# Cliquen
Cliquen sind spezielle Teilmengen innerhalb eines Graphen. Sie können zum Beispiel zur Analyse von sozialen Netzwerken verwendet werden, wenn nach besonders eng vernetzten Teilgruppen gesucht wird. In diesem Abschnitt beschäftigen wir uns mit deren Eigenschaften und Erkennung.

In [ ]:
from itertools import combinations
import networkx as nx

from tui_dsmt.graph import set_label, BronKerbosch
from tui_dsmt.graph.datasets import load_cliques_small

graph = load_cliques_small()

## Inhaltsverzeichnis
- [Definition und Einschränkung](#Definition-und-Einschränkung)
- [Naiver Algorithmus](#Naiver-Algorithmus)
- [Bron-Kerbosch-Algorithmus](#Bron-Kerbosch-Algorithmus)
- [Clique-Percolation-Methode](#Clique-Percolation-Methode)

## Definition und Einschränkung
Eine **Clique** ist eine Teilmenge eines Graphen, die vollständig verbunden ist. In dieser Teilmenge besitzt also jedes Paar von Knoten eine Kante untereinander. (Eine triviale Clique hat daher genau zwei Knoten, die durch eine Kante verbunden sind.)

Eine **maximale Clique** ist eine Clique, die nicht durch Hinzunahme eines weiteren Knoten zu einer größeren Clique werden kann.

Eine **größte Clique** ist eine der Cliquen in einem Graphen, welche die größte Anzahl an Knoten unter allen Cliquen besitzen.

---

In der Regel wird aus mehreren Gründen nach maximalen Cliquen gesucht:
1. Da jede Teilmenge einer maximalen Clique eine Clique ist, wächst die Ergebnismenge sehr schnell.
2. Die Relevanz von kleinen Cliquen ist im Vergleich zu maximalen Cliquen als gering anzusehen.
3. Algorithmen, die nach maximalen Cliquen suchen, können einfacher sein.

## Naiver Algorithmus
Wie immer lässt sich das Problem durch einfaches Probieren lösen:
1. Jeder Knoten kann entweder Teil des Subgraphen sein oder nicht. Eine Clique benötigt mindestens zwei Knoten. (Es gibt also $2^{\left| V \right|} - \left| V \right| - 1$ Kandidaten.)
2. Für jeden Kandidaten wird in absteigender Größe die vollständige Verbundenheit geprüft.
3. Ist der Kandidat eine Clique aber Teil einer bereits gefunden Clique, kann er nicht maximal sein und wird demnach verworfen.

Ein kleiner Beispielgraph, der mehrere Cliquen enthält, sieht wie folgt aus:

In [ ]:
nx.draw(graph, with_labels=True, font_color='whitesmoke')

Der Python-Code unter Verwendung von NetworkX unternimmt nun die oben genannten Schritte:

In [ ]:
# Generierung aller Kandidaten
all_subgraphs = list(set(sub) for size in range(len(graph.nodes), 1, -1) for sub in combinations(graph.nodes, size))
print(f'Anzahl der Subgraphen für {len(graph.nodes)} Knoten: {len(all_subgraphs)}')

# Iteration über alle Kandidaten
result = []

for candidate in all_subgraphs:
    # Herauslösen des Subgraphen
    subgraph = graph.subgraph(candidate)

    # Prüfung auf vollständige Verbundenheit
    if len(subgraph.edges) < len(subgraph.nodes) * (len(subgraph.nodes) - 1) / 2:
        continue

    # Prüfung, ob bereits eine Obermenge enthalten ist
    for previous in result:
        if candidate.issubset(previous):
            break
    else:
        result.append(candidate)

result

Wie üblich skaliert dieser Algorithmus schlecht, weshalb für Graphen in relevanter Größe andere Lösungen gefunden werden müssen.

## Bron-Kerbosch-Algorithmus
Der Bron-Kerbosch-Algorithmus ist einer der bekanntesten Algorithmen, um maximale Cliquen in einem ungerichteten Graphen zu finden. Er wurde von Coenraad Bron und Joep Kerbosch bereits in den 1970er Jahren veröffentlicht und verarbeitet Graphen mit Hilfe von Rekursion und Backtracking.

Der Algorithmus unterteilt die Knoten in drei verschiedene Mengen:
1. $R$ enthält die Knoten der Clique, die zu einer maximalen Clique ausgebaut werden soll. Zu Beginn ist diese Menge leer.
2. $P$ enthält die unerforschten Knoten. Zu Beginn enthält diese Menge alle Knoten des Graphen.
3. $X$ enthält die Menge der bereits ausgeschlossenen Knoten. Zu Beginn ist diese Menge ebenfalls leer.

Ziel des Algorithmus ist es nun eine maximale Clique in $R$ zu sammeln, die einen Teil der Knoten aus $P$ und keinen der Knoten aus $X$ enthält:
- Wenn $P$ und $X$ leer sind, liegt eine maximale Clique in $R$ vor. (Rekursionsabbruch)
- Andernfalls wird über die unerforschten Knoten $v \in P$ iteriert und
    - ein rekursiver Aufruf gestartet, bei dem
        - $v$ zu $R$ hinzugefügt wird,
        - die Nachbarn von $v$ mit $P$ geschnitten werden und
        - die Nachbarn von $v$ mit $X$ geschnitten werden.
    - der Knoten aus $P$ entfernt und $X$ hinzugefügt.

In [ ]:
def bron_kerbosch(graph, R, P, X):
    # Abbruchbedingung
    if len(P) == 0 and len(X) == 0:
        yield R

    # rekursive Aufrufe
    for v in list(P):
        neighbors = set(graph.neighbors(v))
        yield from bron_kerbosch(graph, R.union({v}), P.intersection(neighbors), X.intersection(neighbors))

        P.remove(v)
        X.add(v)

list(bron_kerbosch(graph, set(), set(graph.nodes), set()))

$R$ wird in der nachfolgenden Animation in blau markert, während $P$ grau ist und $X$ rot dargestellt wird. Schwarz sind die Knoten, die in keiner der Mengen enthalten sind. Beachten Sie bitte, dass rekursive "Rückschritte" als einzelne Schritte dargestellt sind.

In [ ]:
BronKerbosch(graph)

## Clique-Percolation-Methode
Die Clique-Percolation-Methode ist ein weiterer Algorithmus zur Identifizierung von Communities in komplexen Netzwerken. Der Algorithmus verwendet Cliquen als Grundlage bei der Suche nach größeren Communities, indem Überlappungen zwischen diesen Cliquen betrachtet werden.

Der Algorithmus arbeitet für einen vorgegebenen Parameter $k$ wie folgt:
- Zuerst werden **alle** Cliquen der Größe $k$ bestimmt.
- Anschließend wird ein Cliquen-Graph erstellt, in dem zwei Cliquen verbunden sind, wenn sie $k-1$ gemeinsame Knoten besitzen.
- Die Komponenten des Graphs repräsentieren die gefundenen Communities.

In [ ]:
# Suche der k-Cliquen
k = 3
k_cliques = [frozenset(c) for c in nx.enumerate_all_cliques(graph) if len(c) == k]

print(k_cliques)

# Erstellen des Clique-Graphen
clique_graph = nx.Graph()

for c1 in k_cliques:
    for c2 in k_cliques:
        if len(c1.intersection(c2)) == k - 1:
            clique_graph.add_edge(set_label(c1), set_label(c2))

nx.draw(clique_graph, with_labels=True, font_size=10)

# Extraction der Komponenten
list(nx.connected_components(clique_graph))